In [24]:
import sys
import os
import transformers
from transformers import pipeline
import json
import torch
import numpy as np
from torch.utils.data import DataLoader , Dataset

In [25]:
sys.path.append("../")

In [26]:
#loading in the JSON list of dictionaries
data = json.load(open("../data/geneset_dict.json"))

In [35]:
print(f"Number of items in geneset_dict: {len(data)}")

Number of items in geneset_dict: 43829


In [30]:
#load in llama model
model_id = "meta-llama/Llama-3.1-70B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [31]:
query = """
Format this data into several question and answer pairs. Use the information from the keys to make multiple questions, 
and the information from the values to make the answer. Prioritize gene id first, then gene name, then aliases, then other names, and finally description.
Format multiple questions to resemble a user query. Make sure you use every key for its own separate question, 
and number the questions. Keep the description as it is. In this case 'aliases' mean synonym symbols for the gene, while 'other names' mean other names for the gene.
Adress every key, but ignore any objects with ['N/A'] as their value. 
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

"""

In [32]:
#function to generate QA pairs from the model
def generate_qa_pairs(sample):
    messages = [
        {"role": "system", "content": query},
        {"role": "user", "content": str(sample)},
    ]
    outputs = pipe(
        messages,
        max_new_tokens=256,
    )
    return outputs[0]["generated_text"][-1]

In [36]:
#to work GPUs parralel
batch_data = [str(item) for item in data[:12]]
outputs = pipe(batch_data, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [40]:
print(outputs)

[[{'generated_text': "{'Symbol': 'A1BG', 'Gene_ID': 'ENSG00000121410', 'Name': 'alpha-1-B glycoprotein', 'Aliases': ['A1B', 'ABG', 'GAB', 'HYST2477'], 'Other names': ['HEL-S-163pA', 'alpha-1B-glycoprotein', 'epididymis secretory sperm binding protein Li 163pA'], 'Description': 'The protein encoded by this gene is a plasma glycoprotein of unknown function. The protein shows sequence similarity to the variable regions of some immunoglobulin supergene family member proteins. [provided by RefSeq, Jul 2008].'} {'Symbol': 'A1BG', 'Gene_ID': 'ENSG00000121410', 'Name': 'alpha-1-B glycoprotein', 'Aliases': ['A1B', 'ABG', 'GAB', 'HYST2477'], 'Other names': ['HEL-S-163pA', 'alpha-1B-glycoprotein', 'epididymis secretory sperm binding protein Li 163pA'], 'Description': 'The protein encoded by this gene is a plasma glycoprotein of unknown function. The protein shows sequence similarity to the variable regions of some immunoglobulin supergene family member proteins. [provided by RefSeq, Jul 2008].'} 

In [39]:
# remove the <question> and <answer> tags from the generated text: copied from chatgpt
final_qa_pairs = []
for result in outputs:
    # Access the 'generated_text' field, which is a string, from the result
    qa_text = result.get('generated_text', '')
    
    # Split the result by <question> and <answer> tags, extract them
    questions_answers = qa_text.split('<question>')
    pair = {}
    for q_a in questions_answers[1:]:
        question_end = q_a.find('</question>')
        answer_end = q_a.find('</answer>')
        question = q_a[:question_end].strip()
        answer = q_a[question_end + len('</question>'):answer_end].strip()
        pair[question] = answer
    final_qa_pairs.append(pair)

with open('qa_pairs.json', 'w') as json_file:
    json.dump(final_qa_pairs, json_file, indent=4)
# print out all the results
print("JSON file saved successfully")

AttributeError: 'list' object has no attribute 'get'